In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Laptop GPU'

In [8]:
from environment import ImageBanditEnv, Experiment
from algorithms import LinUCB, CNN_UCB, ViT_UCB
from pathlib import Path

In [9]:
# f = "env2.pkl"
# if Path(f).is_file():
#     environment = ImageBanditEnv.load(f)
# else:
#     environment = ImageBanditEnv("data.jsonl", 3, 3)
#     environment.save(f)
f = "env3.pkl"
if Path(f).is_file():
    environment = ImageBanditEnv.load(f)
else:
    environment = ImageBanditEnv.from_mnist(Path("dataset"), 3)
    environment.save(f)


In [10]:
linucb = LinUCB(environment)
cnnucb = CNN_UCB(environment, alpha=2)
vitucb = ViT_UCB(environment, lora_r=10, alpha=10)
experiment = Experiment(environment, [linucb, cnnucb, vitucb])

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(cnnucb.numel)
print(vitucb.numel)

28609
12521


In [ ]:
experiment.run(100)

Running :   0%|          | 0/100 [00:00<?, ?it/s]

Running cnnucb:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\wongr\OneDrive\UMD\F25\CMSC473\ViT-Contextual-Bandits\.venv\Lib\site-packages\torch\autograd\graph.py:829: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:179.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [9]:
experiment.generate_results()

In [34]:
[i.mu for i in environment.arms]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [115]:
contexts = environment.get_contexts()
agent2._batch_preds(contexts)

array([5.92799282, 3.86865973, 3.35602832, 5.64981556, 4.93276644,
       4.39350939, 5.87870312, 4.62835598, 7.65383387, 8.00209141])

In [89]:
import numpy as np
from PIL import Image
# for i in np.array(environment.arms[0].images[10]):
#     print(i)
display(environment.arms[8].images[0])
display(Image.open(r"dataset\0\0\0.png"))

In [114]:
agent2.replay.buffer['reward']

deque([9.0, 9.0, 9.0, 1.0, 7.0, 9.0, 9.0, 9.0, 9.0, 9.0], maxlen=10)

deque([0], maxlen=3)
deque([0, 1], maxlen=3)
deque([0, 1, 2], maxlen=3)
deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)
deque([3, 4, 5], maxlen=3)
deque([4, 5, 6], maxlen=3)
deque([5, 6, 7], maxlen=3)
deque([6, 7, 8], maxlen=3)
deque([7, 8, 9], maxlen=3)
